<div align="center" style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding: 10px;">
    <a href="https://github.com/encord-team/text-to-image-eval/actions?query=workflow%3ATests" target="_blank" style="text-decoration:none"><img src="https://github.com/encord-team/text-to-image-eval/actions/workflows/tests.yml/badge.svg?branch=main"></a>
    <a href="https://www.python.org/downloads/release/python-31014/" target="_blank" style="text-decoration:none"><img src="https://img.shields.io/badge/python-3.10%2B-blue" alt="Python Versions"></a>
    <a target="_blank" style="text-decoration:none"><img alt="PRs Welcome" src="https://img.shields.io/badge/PRs-Welcome-blue"></a>
    <img alt="License" src="https://img.shields.io/github/license/encord-team/text-to-image-eval">
  </div>
  <div style="flex: 1; padding: 10px;">
    <a href="https://github.com/encord-team/encord-notebooks" target="_blank" style="text-decoration:none"><img alt="Encord Notebooks" src="https://img.shields.io/badge/Encord_Notebooks-blue?logo=github&label=&labelColor=181717"></a>
    <a href="https://join.slack.com/t/encordactive/shared_invite/zt-1hc2vqur9-Fzj1EEAHoqu91sZ0CX0A7Q" target="_blank" style="text-decoration:none"><img alt="Join us on Slack" src="https://img.shields.io/badge/Join_Our_Community-4A154B?label=&logo=slack&logoColor=white"></a>
    <a href="https://twitter.com/encord_team" target="_blank" style="text-decoration:none"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/encord_team?label=%40encord_team&amp;style=social"></a>
  </div>
</div>

<h1 align="center">
    <img src="https://storage.googleapis.com/docs-media.encord.com/static/img/text-to-image-eval/tti-eval-banner-short.png" alt="tti-eval logo banner">
</h1>

# 📝 tti-eval | 🤖 Bring Your Own Model From Hugging Face Quickstart

Welcome to the _Bring Your Own Model From Hugging Face Quickstart_ guide for the `tti-eval` tool.
This notebook covers how to bring a model from Hugging Face to `tti-eval`.

In this notebook, you will explore:
- How to find the required information of the target model on the Hugging Face website.
- Generating and registering the new model definition.
- ⭐ Start using your model right away.

## Overview

`tti-eval` is a repository for benchmarking text-to-image models **on your own data**!

With this library, you can take an embedding model intended for jointly embedding images and text (like [CLIP](https://huggingface.co/openai/clip-vit-large-patch14-336)) and compute metrics for how well such model performs on classifying your custom dataset.

🟡 Prior to continuing, ensure that Colab's runtime type is set to `T4 GPU` instead of `CPU`. You can find this option under 'Runtime > Change runtime type > Hardware accelerator'.

> Learn more about 📝 `tti-eval`:
> - [🖥️ CLI Quickstart](https://colab.research.google.com/drive/1SsKre_9vQ9SAIQyPFQmbUBXRKzoZv8AH?usp=sharing)
> - [🐍 Python Quickstart](https://colab.research.google.com/drive/1DNn5uH6rss8I4BZG4-Y0rayRkRwYXwMu?usp=sharing)
> - [🗄️ Bring Your Own Dataset From Encord Quickstart](https://colab.research.google.com/drive/1qESU9zx1cBnK3vUDpOeuH6uZjt-f5Cb6?usp=sharing)
> - [GitHub repository](https://github.com/encord-team/text-to-image-eval)

## Set up the repo and dependencies

In [ ]:
!git clone https://github.com/encord-team/text-to-image-eval.git
%cd text-to-image-eval
!python -m pip install -e .

## Bring your model from Hugging Face

Within `tti-eval`, the `HFModel` model type facilitates loading models from Hugging Face.

To register a model from Hugging Face, you will require the full title of the model (`<user>/<model-title>`). With this information, store the model definition as a JSON file in the `sources/models/` folder at the root of the repository, adhering to this structure:

```json
{
  "model_type": "HFModel",
  "title": "<title-in-tti-eval>",
  "title_in_source": "<full-model-title-in-hugging-face>"
}
```

### Find the full title of the model

Open the [Hugging Face website](https://huggingface.co/) and navigate to the target model. The model title is located below the search bar in the top-left corner. Please refer to the image for visual guidance.

![model-title-location-in-hugging-face](https://storage.googleapis.com/docs-media.encord.com/static/img/text-to-image-eval/model-title-location-hugging-face.png)

In [ ]:
#@markdown Overwrite the `title_in_source` variable with the actual model title from Hugging Face
title_in_source = "yainage90/fashion-pattern-clip"

### 2. Register the model to `tti-eval`

 With the full title of the model at hand, let's include the model definition as a JSON file in the `sources/models/` folder at the root of the repository, adhering to this structure:

```json
{
  "model_type": "HFModel",
  "title": "<title-in-tti-eval>",
  "title_in_source": "<full-model-title-in-hugging-face>"
}
```

In [ ]:
import json
from pathlib import Path

model_definition = {
  "model_type": "HFModel",
  "title": "finetuned-fashion",
  "title_in_source": title_in_source,
}

Path('./sources/models/finetuned-fashion.json').write_text(json.dumps(model_definition, indent=2), encoding="utf-8")

Your model has now been permanently incorporated into `tti-eval` and is ready to be utilized for any future tasks.

### [EXTRA] Test the model availability

In [ ]:
#@title Simple test to check that the model is available in the CLI
!tti-eval list --all

Available datasets are: Alzheimer-MRI, chest-xray-classification, LungCancer4Types, plants, rsicd, skin-cancer, sports-classification
Available models are: apple, bioclip, clip, eva-clip, fashion, finetuned-fashion, plip, pubmed, rsicd, rsicd-encord, siglip_large, siglip_small, street, tinyclip, vit-b-32-laion2b


In [ ]:
#@title Let's use Python to get the model
from tti_eval.model import ModelProvider

my_model = ModelProvider.get_model("finetuned-fashion")
my_model.title_in_source

config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

'yainage90/fashion-pattern-clip'

In [ ]:
#@title Advanced test to build the embeddings of a sample dataset
#@markdown It's commented as it can take some time and is slower on a CPU configuration.

!tti-eval build --model-dataset finetuned-fashion/Alzheimer-MRI

Generating train split: 100% 5120/5120 [00:00<00:00, 52293.47 examples/s]
Generating test split: 100% 1280/1280 [00:00<00:00, 74346.50 examples/s]
Embedding (train) Alzheimer-MRI dataset with finetuned-fashion: 100% 88/88 [00:25<00:00,  3.46it/s]
Embeddings saved successfully to file at `/content/text-to-image-eval/.cache/embeddings/Alzheimer-MRI/finetuned-fashion_train.npz`
Embedding (validation) Alzheimer-MRI dataset with finetuned-fashion: 100% 16/16 [00:04<00:00,  3.83it/s]
Embeddings saved successfully to file at `/content/text-to-image-eval/.cache/embeddings/Alzheimer-MRI/finetuned-fashion_validation.npz`


## Wrap Up

This notebook has provided an introduction to bringing a model from Hugging Face to `tti-eval`, showcasing how to find the full title of the relevant model, register the model definition in `tti-eval`, and start using it right away.

---

🟣 `tti-eval` is an open-source repository for benchmarking text-to-image models **on your own data**!
**Check out the project on [GitHub](https://github.com/encord-team/text-to-image-eval) and leave a star 🌟** if you like it.
Contributions are welcome!
Please feel free to open an issue or submit a pull request with your suggestions, bug fixes, or new features.

---

👉 Check out the 📖 [Encord Blog](https://encord.com/blog/) and 📺 [YouTube](https://www.youtube.com/@encord) channel to stay up-to-date with the latest in computer vision, foundation models, active learning, and data-centric AI.


## Next: Learn the how to bring your own dataset from Encord to `tti-eval`.

What's next? 👀 Dive into how to bring a dataset from Encord to `tti-eval`. In the Colab notebook, you will explore:
- How to find the required information of the dataset on the Encord website.
- Generating and registering the new dataset definition.
- ⭐ Start using your dataset right away.

[NEXT NOTEBOOK](https://colab.research.google.com/drive/1qESU9zx1cBnK3vUDpOeuH6uZjt-f5Cb6?usp=sharing) ➡️